In [15]:
import torch 
from train import generate_square_mask

def generate_from_prefix(model, tokenizer, examples, device, prefix, max_len=200, temperature=1.0):
    model.eval()
    for i in range(max_len - prefix.shape[-1]):
        prefix = prefix.clone().detach()
        target_mask = generate_square_mask(prefix.shape[1], device)
        target_pad_mask = (prefix == tokenizer.pad_id()).to(device)

        logits = model(prefix, target_mask, target_pad_mask)
        logits = logits[:, -1, :] / temperature
        next_token = torch.multinomial(logits.softmax(dim=-1), num_samples=1)
        prefix = torch.cat((prefix, next_token), dim=-1)
    return tokenizer.decode(prefix.cpu().tolist())

In [3]:
from data import get_tokenizer
tokenizer = get_tokenizer()

In [6]:
device = "cuda"
checkpoint = torch.load("/notebooks/model.pth", device)

In [9]:
from model import LLaMA
import config

model = LLaMA(config.VOCAB_SIZE, config.EMBED_DIM, config.N_HEADS, config.HIDDEN_DIM, config.NUM_LAYERS, config.MAX_SEQ_LEN).to(device)
model.load_state_dict(checkpoint)


<All keys matched successfully>

In [20]:
prompt = "Once upon a time"
indexes = tokenizer.encode(prompt)
prefix = torch.tensor([[tokenizer.bos_id()] + indexes])
prefix = prefix.to(device)

text = generate_from_prefix(model, tokenizer, 1, "cuda", max_len=256, prefix=prefix)

In [21]:
text

["Once upon a time, there was a little dog named True. True was a patient dog who liked to play and wait for the sun to shine on his back. One day, True found a big piece of gum. He didn't know what it was, so he tried to tell his friends.True's friends saw him trying to resist while playing. They liked gum, so they made it into the sun and put it away when they hid it. Tim felt happy and proud, just like his new trusted his friends. went to show his friends his new friends. was so much fun! now and Max played with their new gum. They all played together, and Max was always happy to have the gum, so they could share it. was happy to be part of the fun he had with it. went on, and Max couldn't wait to play again. made many happy days with their new gum. all by himself, and Max would often let her go of gum. by little and play with him, and they all became good friends. stayed fit, and Max would take shape for the rest of his days. on time, would not feel happy because it. stayed at"]

In [22]:
texts = []
for t in [0.7, 0.8, 0.9, 1, 1.1, 1.2]:
    texts.append(generate_from_prefix(model, tokenizer, 1, "cuda", max_len=256, prefix=prefix, temperature=t))

In [24]:
texts

[['Once upon a time, there was a little boy named Tim. Tim loved to play with his toys and have fun. One day, Tim found a big, round ball that he liked very much. It was red and shiny. He played with it all day long.Tim\'s mom saw him playing with the round ball. She wanted to encourage him to play too. So, Tim went to the window and took the round ball. He saw a big, green light in the sky. He wanted to reach it, but it was too high. Tim had a big smile on his face.Tim\'s mom saw him playing with the red ball. She told him, "Tim, you need to practice. Don\'t go to the moon, he can get it." Tim listened to his mom and practiced. He knew it was important to be careful and not to be too far away from the big, pointy light., and Tim learned that it was not always good to be safe. and Tim played with the big, round ball.s always safe and never let it go. Tim learned that it was better to be safe and listen to his mom.s and never let his play with.s and'],
 ["Once upon a time, there was a l

In [25]:
!pip3 install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 21.7 MB/s eta 0:00:00


In [26]:
import evaluate

perplexity = evaluate.load("perplexity", module_type="metric")

In [27]:
from train import generate

evel_texts = generate(model, tokenizer, 32, device)

In [28]:
results = perplexity.compute(predictions=evel_texts, batch_size=32, model_id='gpt2-large', device=device)

Using pad_token, but it is not set yet.


  0%|          | 0/1 [00:00<?, ?it/s]

In [29]:
results['mean_perplexity']

21.43602228164673

## GPT2-XL

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("gpt2-xl")
model = AutoModelForCausalLM.from_pretrained("gpt2-xl")

In [3]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [4]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2-xl', device=0)

In [5]:
prompt = "Once upon a time"
texts_gpt = generator(prompt, max_length=256, num_return_sequences=16)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [7]:
texts_gpt = list(map(lambda x: x['generated_text'], texts_gpt))

In [8]:
texts_gpt

["Once upon a time, someone wrote a story. A guy named David Foster Wallace did, and you read it, and you knew. A few stories out there now, well, I don't know much about 'em, except they're good, and there's a group of people doing the writing, and it's fun. Maybe there's some kind of relationship, and maybe that relationship exists now. I don't know. So I want to check this out.\n\nI'll start, because I know it's coming from a desire to be in the same place I was when I read Infinite Jest in 2004. At the time, I had already read a big pile of Wallace's work, which included a lot of short fiction. But I read Infinite Jest in a matter of days. I've told stories ever since, because it was so quickly and immediately and totally convincing that I believed it was true.\n\nThere's an enormous irony when you think about how much of a mainstream book you'd been expecting to receive. The story of your own literary career is something that has been almost entirely ignored during your lifetime b

In [12]:
import evaluate

perplexity = evaluate.load("perplexity", module_type="metric")
results_gpt = perplexity.compute(predictions=texts_gpt, batch_size=16, model_id='gpt2-large', device="cuda")

Using pad_token, but it is not set yet.


  0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
results_gpt['mean_perplexity']

11.03925010561943